In [1]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date, timedelta
import yahoo_fin.stock_info as yf

In [2]:
spList = yf.tickers_sp500()
today = date.today()
today = today.strftime('%Y-%m-%d')
dictionary = {}
test = ['AAPL']
for ticker in test:
    try:
        #df = yf.get_data(ticker, start_date="1995-01-01",end_date=today, index_as_date = True, interval="1wk")
        #df.to_pickle('stockData/S&P500/'+ticker+'.pkl')
        #print(ticker)
        if ticker not in dictionary.values():
            industry = yf.get_company_info(ticker).loc['sector'][0]
            if industry not in dictionary.keys():
                dictionary[industry] = []
            dictionary[industry].append(ticker)
    except:
        print("stock not found")

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK-B
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CRL
CRM
CSCO
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CTXS
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDS
FDX
FE
FFIV
FIS
FISV
FITB
FLT
FMC
FOX
FOXA
FRC
FRT
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [4]:
import pickle
try:
    industryDict = open('stockData/industries.pkl', 'wb')
    pickle.dump(dictionary, industryDict)
    industryDict.close()
  
except:
    print("Something went wrong")

In [49]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Discretionary":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(yf.get_data(sector, start_date="1998-12-21",end_date=today, index_as_date = True, interval="1wk"))

In [50]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spDf = yf.get_data('^GSPC', start_date="1998-12-21",end_date=today, index_as_date = True, interval="1wk")
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].close/df.iloc[index-30].close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['WMA30']/df.iloc[index-1]['WMA30']
for df in sectorDfList:
    df['WMA30'] = df['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
    df['WMA30Slope'] = df.apply(lambda x: calculateSlope(df,df.index.get_loc(x.name)),axis=1)
    df['Percent'] = df.apply(lambda x: product(df,df.index.get_loc(x.name)),axis=1)
    spDf['Percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    df['RS'] = df['Percent'] - spDf['Percent']

['AAL', 'ALK', 'DAL', 'LUV', 'UAL']


In [51]:
sectorDfList[0]['RS']

1998-12-21    0.000000
1998-12-28    0.000000
1999-01-04    0.000000
1999-01-11    0.000000
1999-01-18    0.000000
                ...   
2021-12-20   -0.072565
2021-12-27   -0.131270
2022-01-03    0.006479
2022-01-10    0.112200
2022-01-14    0.076430
Name: RS, Length: 1205, dtype: float64

In [52]:
goodSectorDf = sectorDfList[0].drop(['open','high','low','close','adjclose','volume','ticker','WMA30','WMA30Slope','RS','Percent'],axis=1)

In [30]:
for df in sectorDfList:
    df.index = df.index.strftime('%Y-%m-%d')
goodSectorDf.index = sectorDfList[0].index

In [44]:
sectorDfList[0]['RS']

1998-12-21        NaN
1998-12-28        NaN
1999-01-04        NaN
1999-01-11        NaN
1999-01-18        NaN
               ...   
2021-12-20        NaN
2021-12-27        NaN
2022-01-03        NaN
2022-01-10        NaN
2022-01-14    0.07643
Name: RS, Length: 1205, dtype: float64

In [63]:
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
for date in sectorDfList[0].index:
    for df in sectorDfList:
        try:
            ind = df.index.get_loc(date)
            if df.iloc[ind]['WMA30Slope'] > 1 and df.iloc[ind]['RS'] > 0:
                if df.iloc[ind]['close'] > df.iloc[ind]['WMA30']:
                    goodSectorDf.iloc[ind]['Sectors'].append(df.iloc[ind]['ticker'])
        except: 
            pass

In [64]:
from stage import fullPrint
fullPrint(goodSectorDf)

                                              Sectors
1998-12-21                                         []
1998-12-28                                         []
1999-01-04                                         []
1999-01-11                                         []
1999-01-18                                         []
1999-01-25                                         []
1999-02-01                                         []
1999-02-08                                         []
1999-02-15                                         []
1999-02-22                                         []
1999-03-01                                         []
1999-03-08                                         []
1999-03-15                                         []
1999-03-22                                         []
1999-03-29                                         []
1999-04-05                                         []
1999-04-12                                         []
1999-04-19                  

In [68]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
from stage import fullPrint
spDf = yf.get_data('^GSPC', start_date="1995-01-01",end_date=today, index_as_date = True, interval="1wk")
for ticker in spList:
    stockDf = pd.read_pickle('stockData/S&P500/'+ticker+'.pkl')
    stockDf['5YHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
    stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
    stockDf['WMA30Slope'] = stockDf.apply(lambda x: calculateSlope(stockDf,stockDf.index.get_loc(x.name)),axis=1)
    stockDf['VolumePerc'] = stockDf['volume'].pct_change()
    stockDf['Percent'] = stockDf.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
    spDf['Percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    stockDf['RS'] = stockDf['Percent'] - spDf['Percent']
    dfSorted = stockDf.dropna()
    stockDf.to_pickle('stockData/S&P500/'+ticker+'.pkl')